In [1]:
%load_ext autoreload
%autoreload 2

# 前処理

In [2]:
from collections import OrderedDict
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np

import sys
sys.path.append("/home/inoue/work/recs")

from recs.evaluator import metrics, evaluate
from recs import utils
from recs.jaxmodels import gru4rec, GRU4RecConfig
from recs.dataset import session_parallel_dataset
import jax

from flax import serialization

import matplotlib.pyplot as plt
import japanize_matplotlib


2022-11-04 13:00:04.725356: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-04 13:00:05.331151: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-04 13:00:05.331255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-04 13:00:05.331263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make su

In [3]:
#data = pd.read_pickle("~/work/dataset/ml-100k/train.df")
#testdata = pd.read_pickle("~/work/dataset/ml-100k/test.df")

data = pd.read_pickle("~/work/dataset/RC15/derived/train.df")
testdata = pd.read_pickle("~/work/dataset/RC15/derived/test.df")

In [4]:
num_items = max(data.itemId.max(), testdata.itemId.max()) + 1

In [5]:
tscv = TimeSeriesSplit(n_splits=4) 

# GRU4Rec

In [23]:
%%time
batch_size = 500

# hparams = pd.DataFrame(columns=["hidden_size", "dropout_rate", "valid_score"])
count = 0
for hidden_size in [1000]:
    for dropout_rate in [0]:
        # count += 1
        config = GRU4RecConfig(
            hidden_size=hidden_size, 
            output_size=num_items, 
            batch_size=batch_size,
            dropout_hidden=dropout_rate, 
            learning_rate=0.01, 
            final_act="log_softmax",
            num_epochs=10,
            early_stop_count=2
        )

        valid_scores = []
        #for i, (train_ind, valid_ind) in enumerate(tscv.split(data)):
        for i, df in enumerate([data]):
            train_data, total_length = session_parallel_dataset(
                #data.iloc[train_ind],
                data,
                batch_size = config.batch_size,
                issort=False,
                timekey="eventdate"
            )

            # valid_data, valid_len = session_parallel_dataset(
            #     data.iloc[valid_ind],
            #     batch_size=config.batch_size,
            #     issort=False,
            #     timekey="eventdate"
            # )

            state, hidden, model = gru4rec.create_train_state(
                jax.random.PRNGKey(0), config)
            state, hidden, loss = gru4rec.train(
                config,
                model,
                state,
                train_data,
                total_length,
                hidden=hidden
            )

#             valid_pred = gru4rec.predict(config, model, state, valid_data, valid_len)
#             _, valid_score = evaluate.evaluate(
#                 valid_pred,
#                 k=20,
#                 metrics=["recall"]
#             )
#             valid_scores += [valid_score.values[0]]

#         hparams = pd.concat([
#             hparams, 
#             pd.DataFrame([[hidden_size, dropout_rate, np.mean(valid_scores)]], 
#                 columns=["hidden_size", "dropout_rate", "valid_score"])], axis=0)
#         serialization.to_state_dict(o)

[Epoch 2] training:  77%|███████████████████▉      | 647/843 [00:22<00:06, 29.24it/s, loss=5.441398]


KeyboardInterrupt: 

In [22]:
import pickle
pickle.dump(
    serialization.to_state_dict(state.params), open("/home/inoue/work/recs/params/gru4rec_params_%d.pkl"%6, "wb")
)

In [21]:
hparams = hparams.reset_index().drop("index", axis=1)
hparams

,hidden_size,dropout_rate,valid_score
0,100,0,0.163974
1,100,0.2,0.163974
2,100,0.5,0.163974
3,500,0,0.066886
4,500,0.2,0.066886
5,500,0.5,0.066886
6,1000,0,0.03697
7,1000,0.2,0.03697


In [23]:
hparams.to_csv("/home/inoue/work/recs/params/gru4rec_hparams.csv", index=False)

In [6]:
batch_size = 500
hidden_size = 1000
dropout_rate=0

config = GRU4RecConfig(
    hidden_size=hidden_size, 
    output_size=num_items, 
    batch_size=batch_size,
    dropout_hidden=dropout_rate, 
    learning_rate=0.01, 
    final_act="log_softmax",
    num_epochs=10,
    early_stop_count=2
)

train_data, total_length = session_parallel_dataset(
    data,
    batch_size = config.batch_size,
    issort=False,
    #sessionkey="userId",
    #timekey="timestamp"
)

state, hidden, model = gru4rec.create_train_state(
    jax.random.PRNGKey(0), config)
state, hidden, loss = gru4rec.train(
    config,
    model,
    state,
    train_data,
    total_length,
    hidden=hidden
)

[Epoch 1] training:  39%|████████▌             | 18462/47456 [08:30<13:20, 36.20it/s, loss=7.786817]


KeyboardInterrupt: 

In [1]:
test_data, test_len = session_parallel_dataset(
    testdata,
    batch_size=config.batch_size, 
    issort=False,
    #sessionkey="userId",
    #timekey="timestamp"
)

NameError: name 'session_parallel_dataset' is not defined

In [17]:
testdata["userId"].max()

942

In [8]:
pred_df = gru4rec.predict(
    config,
    model,
    state,
    test_data,
    test_len,
)

predicting:   0%|                                                            | 0/39 [00:00<?, ?it/s]2022-11-02 17:45:28.652051: W tensorflow/core/framework/op_kernel.cc:1768] UNKNOWN: IndexError: index 301 is out of bounds for axis 0 with size 301
Traceback (most recent call last):

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/inoue/work/recs/recs/dataset.py", line 33, in generator
    start, end = click_offsets[session_idx_arr[iters]], click_offsets[session_idx_arr[iters] + 1]

IndexError: index 301 is out of bounds for axis 0 with size 301


predicting:   0%|     

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} IndexError: index 301 is out of bounds for axis 0 with size 301
Traceback (most recent call last):

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/inoue/work/recs/recs/dataset.py", line 33, in generator
    start, end = click_offsets[session_idx_arr[iters]], click_offsets[session_idx_arr[iters] + 1]

IndexError: index 301 is out of bounds for axis 0 with size 301


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

In [12]:
eval_scores = pd.DataFrame(columns=["precision", "ndcg"])
for k in [5, 10, 20]:
    _, evals = evaluate.evaluate(
        pred_df,
        k=k,
        metrics=["precision", "ndcg"]
    )
    eval_scores = pd.concat([
        eval_scores,
        evals
    ], axis=0)    

# GRU4Rec-SQN

In [5]:
from recs.jaxmodels import gru4rec_sqn

In [6]:
batch_size = 500

hparams = pd.DataFrame(columns=["hidden_size", "dropout_rate", "valid_score"])

for hidden_size in [100]:
    for dropout_rate in [0.]:
        config = GRU4RecConfig(
            hidden_size=hidden_size, 
            output_size=num_items, 
            batch_size=batch_size,
            dropout_hidden=dropout_rate, 
            learning_rate=0.001, 
            final_act="log_softmax",
            num_epochs=500,
            early_stop_count=3,
            gamma=1.
        )

        valid_scores = []
        #for i, (train_ind, valid_ind) in enumerate(tscv.split(data)):
        for i, df in enumerate([data]):
            print("Split %d"%(i+1))
            train_data, total_length = session_parallel_dataset(
                #data.iloc[train_ind],
                data,
                batch_size = config.batch_size,
                issort=False,
                timekey="eventdate"
            )

            state, hidden, model = gru4rec_sqn.create_train_state(
                jax.random.PRNGKey(0), config)
            state, hidden, loss = gru4rec_sqn.train(
                config,
                model,
                state,
                train_data,
                total_length,
                hidden=hidden
            )

Split 1


[Epoch 357] training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 843/843 [00:10<00:00, 79.10it/s, loss=1.356113]


In [7]:
test_data, test_len = session_parallel_dataset(
    testdata,
    batch_size=config.batch_size, 
    issort=False,
    timekey="eventdate"
)

[autoreload of recs.jaxmodels.nn.gru4rec_sqn failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__ = new
  File "/usr/local/lib/python3.8/dist-packages/flax/linen/module.py", line 700, in __setattr__
    raise errors.SetAttributeFrozenModuleError(
flax.errors.SetAttributeFrozenModuleError: Can't set __class__=<class 'recs.j

In [9]:
pred_df = gru4rec_sqn.predict(
    config,
    model,
    state,
    test_data,
    test_len,
)

[autoreload of recs.jaxmodels.nn.gru4rec_sqn failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/usr/local/lib/python3.8/dist-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__ = new
  File "/usr/local/lib/python3.8/dist-packages/flax/linen/module.py", line 700, in __setattr__
    raise errors.SetAttributeFrozenModuleError(
flax.errors.SetAttributeFrozenModuleError: Can't set __class__=<class 'recs.j

In [22]:
unique_items = pred_df.groupby("sessionId")["trueIds"].unique()

In [26]:
df = pd.merge(pred_df, unique_items.reset_index(name="true_list"), on="sessionId", how="left")

In [27]:
df

,sessionId,predIds,trueIds,true_list
0,2296,"[15077, 34715, 4563, 18460, 2575, 16269, 14100...",2208,"[2208, 13385]"
1,2299,"[11261, 35238, 38847, 25835, 33507, 938, 23172...",9649,"[9649, 37623]"
2,2302,"[31579, 25914, 35238, 40559, 1102, 3085, 36543...",28766,"[28766, 31346]"
3,2311,"[25769, 36320, 36063, 4729, 10797, 11761, 6287...",12316,"[12316, 3279]"
4,2308,"[12908, 4404, 12912, 3670, 20306, 37309, 9915,...",24310,"[24310, 15092, 13403]"
...,...,...,...,...
142995,44461,"[33003, 9973, 4475, 18123, 6172, 27062, 1087, ...",22705,"[28356, 22705]"
142996,88157,"[9424, 11124, 9474, 1680, 3453, 4684, 11201, 3...",9018,[9018]
142997,44018,"[12037, 11261, 8494, 5752, 19466, 24727, 12154...",21981,[21981]
142998,29791,"[9651, 6728, 21316, 16276, 6621, 15494, 17837,...",9004,"[1650, 29144, 9004]"


In [36]:
df[df["sessionId"] == 0].index

Int64Index([109313, 109813, 110313], dtype='int64')

In [37]:
from tqdm.notebook import tqdm
for i in tqdm(df["sessionId"].unique()):
    ind = df[df["sessionId"] == i].index
    for n, j in enumerate(ind):
        df.loc[j, "true_list"] = df.loc[j, "true_list"][n:]


  0%|          | 0/61668 [00:00<?, ?it/s]

In [40]:
#df = df.drop("trueIds", axis=1)
df = df.rename({"true_list":"trueIds"}, axis=1)

In [49]:
evals, eval_scores = evaluate.evaluate(
    df,
    k=20,
    metrics=["recall", "ndcg"]
)

In [50]:
eval_scores

recall    0.131633
ndcg      0.004110
dtype: float64

# GRU fori_loop

In [5]:
from typing import Optional, Tuple
from collections import OrderedDict
from tqdm import tqdm

import numpy as np
import pandas as pd

import jax
from jax import numpy as jnp

import optax
from flax import struct
from flax import linen as nn
from flax.training import train_state
from flax.training import common_utils

In [6]:
class GRU4Rec(nn.Module):
    config : GRU4RecConfig
    @staticmethod
    def init_hidden( 
        batch_size:int,
        hidden_size:int,
        num_layers:int,
        rng_key:Optional[int]=0
    ):
        return nn.GRUCell().initialize_carry(
                jax.random.PRNGKey(rng_key), 
                (batch_size,),
                hidden_size*num_layers).reshape((num_layers, -1, hidden_size))
    
    def final_act(self):
        if self.config.final_act == "softmax":
            return nn.softmax
        elif self.config.final_act == "tanh":
            return nn.tanh
        elif self.config.final_act == "relu":
            return nn.relu
        elif self.config.final_act == "log_softmax":
            return nn.log_softmax
    
    @nn.compact
    def __call__(
        self,
        inputs:jnp.ndarray,
        hidden:jnp.ndarray,
        deterministic:Optional[bool]=True
    ):
        """
        Args :
            inputs (batch_size,) : 
                    batch of item indices from a session-paralell mini-batch
            hidden (num_layers, batch_size, hidden_size):
            deterministic:

        """
        hidden_list = []
        if self.config.embedding_dim > 0:
            x = nn.Embed(self.config.output_size, self.config.embedding_dim)(inputs)
        else:
            x = jax.nn.one_hot(inputs, self.config.output_size)
        
        for i in range(self.config.num_layers):
            hidden_, x = nn.GRUCell()(hidden[i], x)
            x = nn.Dropout(self.config.dropout_hidden)(x, deterministic=deterministic)
            hidden_list += [hidden_]
        
        
        output = nn.Dense(self.config.output_size)(x)
        output = self.final_act()(output)
        return output, jnp.stack(hidden_list, axis=0)


In [7]:
def create_train_state(key, config):
    model = GRU4Rec(config)
    hidden = model.init_hidden(config.batch_size, config.hidden_size, config.num_layers)
    params = model.init(key, jnp.ones((config.batch_size,), dtype=jnp.int32), hidden)["params"]
    tx = optax.adam(config.learning_rate)
    return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=tx), hidden, model

In [8]:
def cross_entropy_loss(
    y_true:jnp.ndarray, # (batch_size, )
    y_pred:np.ndarray   # (batch_size, num_items)
):
    
    logits = y_pred[:, y_true]

    one_hot_labels = jax.nn.one_hot(
        jnp.arange(y_true.shape[0]), 
        num_classes=y_true.shape[0])

    loss = -jnp.mean(jnp.sum(one_hot_labels * logits, axis=1))
    return loss

In [9]:
@jax.jit
def train_step(state, x, y, mask, hidden):
        
    if len(mask) != 0:
        hidden = hidden.at[:, mask, :].set(0)

    def loss_fn(params, hidden):
        logits, hidden = model.apply({"params":params}, x, hidden) # (batch_size, num_items)
        logits, targets = jnp.squeeze(logits), jnp.squeeze(y)            
        loss = cross_entropy_loss(targets, logits)
        return loss, hidden
    (loss, hidden), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params, hidden)
    state = state.apply_gradients(grads=grads)
    return state, loss, hidden

In [11]:
hidden_size = 100
batch_size=500
dropout_rate=0

config = GRU4RecConfig(
            hidden_size=hidden_size, 
            output_size=num_items, 
            batch_size=batch_size,
            dropout_hidden=dropout_rate, 
            learning_rate=0.01, 
            final_act="log_softmax",
            num_epochs=10,
            early_stop_count=2)

#state = create_train_state(jax.random.PRNGKey(0), config)